# Imports

In [1]:
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import load, save
import mathphys.constants as cons
from pycolleff.longitudinal_equilibrium import ImpedanceSource, LongitudinalEquilibrium

from pycolleff.rings import maxiv, als, half, sirius

from pycolleff.longitudinal_equilibrium import _mytrapz

from pycolleff.colleff import Ring

import matplotlib as mplt
import matplotlib.cm as cmap

import scienceplots
from itertools import product

mplt.style.use('science')

plt.rcParams.update({
    'font.size': 18, 'axes.grid': True,
    'grid.linestyle': '--', 'grid.alpha': 0.5})

plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.right'] = False
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['xtick.top'] = False

c = 299_792_458

# Methods

In [2]:
def calc_equilibrium_for_fixed_detuning(
    ring, hcav, current, detune, longeq=None
):
    """Calculate the equilibrium for a fixed cavity and current."""
    h = ring.harm_num    
    ring.total_current = current
    fill = np.ones(h) *  ring.total_current/h
    
    if longeq is None:
        longeq = LongitudinalEquilibrium(
            ring=ring, impedance_sources=[hcav, ],
            fillpattern=fill)
        longeq.feedback_on = False
        longeq.zgrid = np.linspace(-1, 1, 2001) * ring.rf_lamb/2
        # longeq.zgrid = np.linspace(-0.9, 0.9, 2001) * ring.rf_lamb/2
    else:
        longeq.ring = ring
        longeq.impedance_sources = [hcav]
        longeq.ring.total_current = current
        longeq.fillpattern = fill
        
    longeq.impedance_sources[0].detune_w = 2*np.pi*detune
    _ = longeq.calc_longitudinal_equilibrium(
            niter=1_000, tol=1e-8, beta=0.1, print_flag=True, m=3)
    return longeq


def calc_equilibrium_for_flat_potential(
    ring, hcav, current, nr_iters=7, longeq=None
):
    """Calculate equilibrium to flat potential condition.

    This method will change the detune_angle of the cavity so that
    the beam induced voltage amplitude matches the flat potential
    condition.

    It will iterate a few times so that the form factor of the
    distribution converges.
    """
    h = ring.harm_num    
    fill = np.ones(h) * current/h
    ring.total_current = current
    
    if longeq is None:
        longeq = LongitudinalEquilibrium(
            ring=ring, impedance_sources=[hcav, ],
            fillpattern=fill)
        longeq.feedback_on = False
        longeq.zgrid = np.linspace(-1, 1, 2001) * ring.rf_lamb/2
        # longeq.zgrid = np.linspace(-0.9, 0.9, 2001) * ring.rf_lamb/2
    else:
        longeq.ring = ring
        longeq.impedance_sources = [hcav]
        longeq.ring.total_current = current
        longeq.fillpattern = fill
    
    for _ in range(nr_iters):
        kharm = longeq.calc_harmonic_voltage_for_flat_potential()
        vharm = kharm * longeq.ring.gap_voltage
        angle = longeq.calc_detune_for_fixed_harmonic_voltage(
            vharm, Rs=hcav.shunt_impedance)
        hcav.detune_angle = angle
        longeq.impedance_sources[0] = hcav
        print(f'detune: {hcav.detune_w/2/np.pi/1e3:.3f} kHz')

        _ = longeq.calc_longitudinal_equilibrium(
                niter=1000, tol=1e-8, beta=0.1, print_flag=True, m=3)
    return longeq


################################################################################
def calc_lmci(
    longeq,
    cbmode=1,
    max_azi=2,
    max_rad=1,
    fokker=True,
    delete_m0=True,
    delete_m0k0=True,
    sync_method='sigma',
):
    """Calculate LMCI for a given coupled bunch mode and equilibrium condition.

    Args:
        longeq: object from Longitudinal equilibrium class;
        cbmode: coupled-bunch mode to evaluate must be in range [0, h-1].
            Defaults to 1, which is the mode of PTBL instability;
        max_azi: maximum azimuthal mode to consider in expansion.
            Detaults to 2. Found to be enough to explain PTBL instability;
        max_azi: maximum radial mode to consider in expansion. Defaults to 1.
            Also found to be enough to explain PTBL instability.

    Returns:
        eigenfreq: complex coherent eigen angular frequencies in [rad/s];
        freqshift: frequency of the most unstable mode in [Hz];
        grate: growth rate of the most unstable mode in [1/s];
        sync_freq: incoherent synchrotron frequency used in calculations [Hz];
        peak_hvolt: peak voltage of harmonic cavity [V].
    
    """   
    hcav = longeq.impedance_sources[0]
    hvolt = longeq.calc_induced_voltage_wake(
        dist=longeq.distributions, wake_source=hcav)
    mvolt = longeq.main_voltage
    tvolt = mvolt + hvolt
    peak_hvolt = np.max(np.abs(hvolt[0, :]))
    peak_mvolt = np.max(np.abs(mvolt))

    print(f'current: {longeq.ring.total_current*1e3:.2f} mA')
    print(f'detune: {hcav.detune_w/2/np.pi/1e3:.3f} kHz')
    print(f'detune angle: {np.rad2deg(hcav.detune_angle):.1f} deg')
    print(f'R/Q: {hcav.shunt_impedance/hcav.Q:.1f} Ohm')
    print(f'HC voltage: {peak_hvolt/1e3:.1f} kV')
    ratio = peak_hvolt/peak_mvolt
    print(f'HC voltage ratio: {ratio:.3f}')

    wrf = longeq.ring.rf_ang_freq
    dist, pot = longeq.calc_distributions_from_voltage(voltage=tvolt)
    z0, sigmaz = longeq.calc_moments(longeq.zgrid, dist[0])
    sigmaz = sigmaz[0]
    longeq.ring.bunlen = sigmaz

    # Calculate the equivalent synchrotron frequency:
    if sync_method.startswith('sigma'):
        # use fórmula based on bunch length:
        sync_freq = longeq.ring.espread * longeq.ring.mom_comp * c
        sync_freq /= sigmaz * 2 * np.pi
    elif sync_method.startswith('average'):
        # use average of distribution:
        sync_freq = longeq.calc_synchrotron_frequency(
            tvolt, method="action", max_amp=5, nrpts=100
        )['avg_sync_freq']
    elif sync_method.startswith('derivative'):
        # use average of square root of gap voltage:
        sync_freq = longeq.calc_synchrotron_frequency(
            tvolt, method="derivative", max_amp=5, nrpts=100
        )['avg_sync_freq']
    longeq.ring.sync_tune = sync_freq / longeq.ring.rev_freq
    
    print(f"sync. freq.: {longeq.ring.sync_tune * longeq.ring.rev_freq:.3f} Hz")
    print(f'bunch length: {longeq.ring.bunlen*1e3:.3f} mm')

    # Calculate Vlasov's equation eigen-frequencies:
    wrf = longeq.ring.rf_ang_freq
    reduced = False
    eigenfreq, *_ = longeq.calc_mode_coupling(
        w=[-10*wrf, +10*wrf], 
        cbmode=cbmode,
        max_azi=max_azi,
        max_rad=max_rad, 
        use_fokker=fokker,
        reduced=reduced,
        delete_m0=delete_m0,
        delete_m0k0=delete_m0k0,
    )

    # Find most unstable mode:
    idx = np.argmax(eigenfreq.imag)
    freqshift = eigenfreq.real[idx] / 2 / np.pi
    grate = eigenfreq.imag[idx]
    if reduced:
        # Subtract radiation damping rate from "reduced" simulation:
        grate -= 1/longeq.ring.dampte

    print('Most unstable mode:')
    print(f'    frequency: {freqshift:.2f} Hz')
    print(f'    growth rate: {grate:.2f} 1/s')
    print('\n')
    return eigenfreq, freqshift, grate, sync_freq, peak_hvolt


def calc_lmci_scan_current(ring, hcav, current_scan):
    """Calculate equilibrium and solve Vlasov equation for a current set.

    The cavity will be detuned to flat potential condition at each current.
    
    """
    growths, shifts, syncdata, peak_hvolts = [], [], [], []
    longeq = None
    cbmode = 1
    # cbpmode = ring.harm_num-1
    for i, current in enumerate(current_scan):
        longeq = calc_equilibrium_for_flat_potential(
            ring, hcav, current=current, nr_iters=8, longeq=longeq,
        )
        eigenfreq, tuneshift, grate, sync_freq, peak_hvolt = calc_lmci(
            longeq, cbmode=cbmode, max_azi=3, max_rad=2,
        )
        growths.append(grate)
        shifts.append(tuneshift)
        syncdata.append(sync_freq)
        peak_hvolts.append(peak_hvolt)
    growths = np.array(growths)
    shifts = np.array(shifts)
    syncdata = np.array(syncdata)
    peak_hvolts = np.array(peak_hvolts)
    return growths, shifts, syncdata, peak_hvolts


def calc_lmci_scan_detuning(
    ring,
    hcav,
    current,
    detuning_scan,
    fokker=True,
    delete_m0=True,
    delete_m0k0=True,
    sync_method='sigma',
):
    """Calculate equilibrium and solve Vlasov equation for a detuning set."""
    growths, shifts, syncdata, peak_hvolts = [], [], [], []
    longeq = None
    cbmode = 1
    # cbpmode = ring.harm_num-1
    for i, detune in enumerate(detuning_scan):
        longeq = calc_equilibrium_for_fixed_detuning(
            ring, hcav, current, detune=detune, longeq=longeq,
        )
        eigenfreq, tuneshift, grate, sync_freq, peak_hvolt = calc_lmci(
            longeq,
            cbmode=cbmode,
            max_azi=3,
            max_rad=2,
            fokker=fokker,
            delete_m0=delete_m0,
            delete_m0k0=delete_m0k0,
            sync_method=sync_method,
        )
        growths.append(grate)
        shifts.append(tuneshift)
        syncdata.append(sync_freq)
        peak_hvolts.append(peak_hvolt)
    growths = np.array(growths)
    shifts = np.array(shifts)
    syncdata = np.array(syncdata)
    peak_hvolts = np.array(peak_hvolts)
    return growths, shifts, syncdata, peak_hvolts

# MAX-IV

In [3]:
ring = maxiv.create_ring(energy=3)
ring.gap_voltage = 1.397e6
ring.total_current = 300e-3

print(ring)

Lattice Version             :     MAX-IV-3GeV     
Circumference [m]           :       527.999       
Revolution Period [us]      :        1.761        
Revolution Frequency [kHz]  :       567.790       
Energy [GeV]                :        3.000        
U0 [keV]                    :       363.800       
Vgap [MV]                   :        1.397        
Momentum Compaction         :       3.06e-04      
Harmonic Number             :         176         
Current [mA]                :       300.000       
Current per Bunch [mA]      :        1.705        
Synchrotron Tune            :       0.00164       
Tunes x/y                   :    16.280/42.200    
Chromaticities x/y          :     1.000/1.000     
Damping Times x/y/e [ms]    :   15.7/ 29.0 /25.2  
Energy Spread [%]           :        0.0769       
Bunch Length [mm]           :        10.694       



In [4]:
hcav = ImpedanceSource()
hcav.harm_rf = 3
hcav.Q = 20_800
ncavs = 3
hcav.shunt_impedance = 2.75e6 * ncavs 

twopi = 2*np.pi
hcav.ang_freq_rf = twopi*ring.rf_freq
hcav.ang_freq = hcav.harm_rf*hcav.ang_freq_rf

hcav.detune_w = twopi*25e3
hcav.calc_method = ImpedanceSource.Methods.ImpedanceDFT
hcav.active_passive = ImpedanceSource.ActivePassive.Passive

In [14]:
current_scan = np.linspace(250, 400, 30)*1e-3
grate, freq, sync_freq, peak_hvolt = calc_lmci_scan_current(ring, hcav, current_scan)

detune: 65.903 kHz
Iter.: 001, Dist. Diff.: 1.967e+00 (bucket 000), E.T.: 0.038s
--------------------
Iter.: 002, Dist. Diff.: 1.930e+00 (bucket 000), E.T.: 0.047s
--------------------
Iter.: 003, Dist. Diff.: 1.911e+00 (bucket 005), E.T.: 0.047s
--------------------
Iter.: 004, Dist. Diff.: 1.729e+00 (bucket 000), E.T.: 0.052s
--------------------
Iter.: 005, Dist. Diff.: 1.489e+00 (bucket 005), E.T.: 0.048s
--------------------
Iter.: 006, Dist. Diff.: 1.619e+00 (bucket 005), E.T.: 0.049s
--------------------
Iter.: 007, Dist. Diff.: 1.657e+00 (bucket 005), E.T.: 0.048s
--------------------
Iter.: 008, Dist. Diff.: 1.628e+00 (bucket 005), E.T.: 0.048s
--------------------
Iter.: 009, Dist. Diff.: 1.459e+00 (bucket 005), E.T.: 0.050s
--------------------
Iter.: 010, Dist. Diff.: 1.397e+00 (bucket 005), E.T.: 0.049s
--------------------
Iter.: 011, Dist. Diff.: 1.238e+00 (bucket 005), E.T.: 0.051s
--------------------
Iter.: 012, Dist. Diff.: 1.105e+00 (bucket 005), E.T.: 0.049s
------

In [15]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ay = ax.twinx()

ax.plot(current_scan*1e3, freq, 'o-')
ax.plot(current_scan*1e3, sync_freq, color='k', label=r'$f_0\nu_s$')
ay.plot(current_scan*1e3, np.array(grate), 'o-', color='tab:red')

ay.grid(False)
ay.spines['right'].set_color('tab:red')
# plt.setp(ay.yaxis.get_ticklabels(), color='tab:red')
ay.tick_params(axis='y', colors='tab:red')

ay.spines['left'].set_color('C0')
plt.setp(ax.yaxis.get_ticklabels(), color='tab:red')
ax.tick_params(axis='y', colors='C0')

ax.legend(loc='center', frameon=True)
ax.set_ylabel('Frequency [Hz]', color='C0')
ax.set_xlabel('Currents [mA]')
ay.set_ylabel('Growth Rate [1/s]', color='tab:red')
fig.tight_layout()
fig.show()

In [16]:
current = 300e-3
detune_scan = np.linspace(77.5, 73, 20)*1e3
# detune_scan = detune_scan[:14]
fokker = True
delete_m0 = True
delete_m0k0 = True
sync_method = 'average'
grate, freq, sync_freq, peak_hvolt = calc_lmci_scan_detuning(
    ring,
    hcav,
    current,
    detune_scan,
    fokker=fokker,
    delete_m0=delete_m0,
    delete_m0k0=delete_m0k0,
    sync_method=sync_method,
)

Iter.: 001, Dist. Diff.: 1.956e+00 (bucket 000), E.T.: 0.040s
--------------------
Iter.: 002, Dist. Diff.: 1.924e+00 (bucket 000), E.T.: 0.039s
--------------------
Iter.: 003, Dist. Diff.: 1.904e+00 (bucket 000), E.T.: 0.039s
--------------------
Iter.: 004, Dist. Diff.: 1.709e+00 (bucket 000), E.T.: 0.040s
--------------------
Iter.: 005, Dist. Diff.: 1.508e+00 (bucket 000), E.T.: 0.042s
--------------------
Iter.: 006, Dist. Diff.: 1.621e+00 (bucket 000), E.T.: 0.045s
--------------------
Iter.: 007, Dist. Diff.: 1.683e+00 (bucket 000), E.T.: 0.054s
--------------------
Iter.: 008, Dist. Diff.: 1.682e+00 (bucket 000), E.T.: 0.041s
--------------------
Iter.: 009, Dist. Diff.: 1.689e+00 (bucket 000), E.T.: 0.048s
--------------------
Iter.: 010, Dist. Diff.: 1.572e+00 (bucket 000), E.T.: 0.050s
--------------------
Iter.: 011, Dist. Diff.: 1.025e+00 (bucket 000), E.T.: 0.047s
--------------------
Iter.: 012, Dist. Diff.: 9.084e-01 (bucket 003), E.T.: 0.058s
--------------------
Iter

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ay = ax.twinx()

hvol = peak_hvolt / 448e3
ax.plot(hvol, freq, 'o-')
ax.plot(hvol, sync_freq, 'c', label=r'$f_0\nu_s$')
ay.plot(hvol, grate, 'o-', color='tab:red')
ax.axvline(
    hvol[(grate>0).nonzero()[0][0]],
    ls='--',
    color='k',
    label='Threshold',
)

ay.grid(False)
ay.spines['right'].set_color('tab:red')
# plt.setp(ay.yaxis.get_ticklabels(), color='tab:red')
ay.tick_params(axis='y', colors='tab:red')

ay.spines['left'].set_color('C0')
plt.setp(ax.yaxis.get_ticklabels(), color='tab:red')
ax.tick_params(axis='y', colors='C0')

ax.set_title(
    f'Fokker = {fokker}, delete_m0 = {delete_m0}, sync_method = {sync_method:s}')

ax.legend(loc='lower left', frameon=True, bbox_to_anchor=(0.05, 0.15))
ax.set_ylabel('Frequency [Hz]', color='C0')
ax.set_xlabel(r'Harmonic Voltage / $V_\text{flat}$')
# ax.set_xlabel('Harmonic Voltage [kV]')
ay.set_ylabel('Growth Rate [1/s]', color='tab:red')
fig.tight_layout()
fig.show()

# Test Area

In [23]:
longeq = calc_equilibrium_for_flat_potential(ring, hcav, 300e-3, nr_iters=7)

detune: 79.230 kHz
Iter.: 001, Dist. Diff.: 1.962e+00 (bucket 000), E.T.: 0.043s
--------------------
Iter.: 002, Dist. Diff.: 1.927e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 003, Dist. Diff.: 1.907e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 004, Dist. Diff.: 1.719e+00 (bucket 000), E.T.: 0.047s
--------------------
Iter.: 005, Dist. Diff.: 1.497e+00 (bucket 000), E.T.: 0.048s
--------------------
Iter.: 006, Dist. Diff.: 1.615e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 007, Dist. Diff.: 1.656e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 008, Dist. Diff.: 1.634e+00 (bucket 000), E.T.: 0.051s
--------------------
Iter.: 009, Dist. Diff.: 1.542e+00 (bucket 000), E.T.: 0.048s
--------------------
Iter.: 010, Dist. Diff.: 1.479e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 011, Dist. Diff.: 1.611e+00 (bucket 000), E.T.: 0.046s
--------------------
Iter.: 012, Dist. Diff.: 9.535e-01 (bucket 000), E.T.: 0.046s
------

In [24]:
hvolt = longeq.calc_induced_voltage_wake(
    dist=longeq.distributions, wake_source=hcav)

In [32]:
fig, ax = plt.subplots(1, 1)

ax.plot(longeq.zgrid, hvolt[0])
fig.tight_layout()
fig.show()

In [12]:
detune_scan[14]

74184.2105263158

In [13]:
longeq = calc_equilibrium_for_fixed_detuning(ring, hcav, 300e-3, detune_scan[15])

Iter.: 001, Dist. Diff.: 1.941e+00 (bucket 000), E.T.: 0.039s
--------------------
Iter.: 002, Dist. Diff.: 1.915e+00 (bucket 000), E.T.: 0.040s
--------------------
Iter.: 003, Dist. Diff.: 1.896e+00 (bucket 002), E.T.: 0.040s
--------------------
Iter.: 004, Dist. Diff.: 1.670e+00 (bucket 000), E.T.: 0.040s
--------------------
Iter.: 005, Dist. Diff.: 1.576e+00 (bucket 000), E.T.: 0.043s
--------------------
Iter.: 006, Dist. Diff.: 1.663e+00 (bucket 000), E.T.: 0.042s
--------------------
Iter.: 007, Dist. Diff.: 1.718e+00 (bucket 000), E.T.: 0.051s
--------------------
Iter.: 008, Dist. Diff.: 1.724e+00 (bucket 000), E.T.: 0.043s
--------------------
Iter.: 009, Dist. Diff.: 1.723e+00 (bucket 004), E.T.: 0.044s
--------------------
Iter.: 010, Dist. Diff.: 1.759e+00 (bucket 002), E.T.: 0.046s
--------------------
Iter.: 011, Dist. Diff.: 1.679e+00 (bucket 000), E.T.: 0.043s
--------------------
Iter.: 012, Dist. Diff.: 1.502e+00 (bucket 000), E.T.: 0.043s
--------------------
Iter

In [14]:
pot = longeq.calc_induced_voltage_wake(hcav)
pot += longeq.main_voltage

fig, ax = plt.subplots(1, 1)

ax.plot(longeq.zgrid, pot[0])
fig.show()